In [3]:
import sqlite3

In [5]:
path = ''
db_name = 'assignment.db'
conn = sqlite3.connect(path + db_name)
conn.close()

In [6]:
path = ''
db_name = 'assignment.db'
conn = sqlite3.connect(path + db_name)
cur = conn.cursor()
# Github上でGoogleが管理しているリポジトリの情報をスクレイピングしてリポジトリ名、主要な言語、スターの数をGithubのAPIを使わずに取得し、SQLiteデータベースに保存する
url = 'https://github.com/google?tab=repositories'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
repo_list = soup.find_all('li', class_='public source')
for repo in repo_list:
    name = repo.find('a', itemprop='name codeRepository').text.strip()
    language_tag = repo.find('span', itemprop='programmingLanguage')
    language = language_tag.text.strip() if language_tag else 'N/A'
    stars_tag = repo.find('a', class_='Link--muted d-inline-block mr-3')
    stars = stars_tag.text.strip() if stars_tag else '0'
    sql = "INSERT INTO repositories (name, language, stars) VALUES (?, ?, ?);"
    cur.execute(sql, (name, language, stars))
conn.commit()
conn.close()

NameError: name 'requests' is not defined